<a href="https://colab.research.google.com/github/muajnstu/Bank-Marketing-using-rough-set-approach/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q imbalanced-learn catboost xgboost lightgbm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, matthews_corrcoef, classification_report, roc_curve, auc
)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso, ElasticNet, Perceptron
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier,
    ExtraTreesClassifier, StackingClassifier, BaggingClassifier, VotingClassifier
)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier

base_estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ('lgbm', LGBMClassifier(random_state=42))
]

import warnings
import warnings
warnings.filterwarnings("ignore", message="'force_all_finite' was renamed to 'ensure_all_finite'")
warnings.filterwarnings("ignore", message="Parameters: { \"use_label_encoder\" } are not used.")


In [ ]:
def g_mean(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    if cm.shape == (2, 2):  # binary classification
        sensitivity = cm[1,1] / (cm[1,0] + cm[1,1]) if (cm[1,0] + cm[1,1]) != 0 else 0
        specificity = cm[0,0] / (cm[0,0] + cm[0,1]) if (cm[0,0] + cm[0,1]) != 0 else 0
        return np.sqrt(sensitivity * specificity)
    else:
        return np.nan

In [3]:
classifiers = {
    "Logistic Regression": LogisticRegression(),
    "SVM": SVC(kernel='rbf'),
    "Naive Bayes": GaussianNB(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "LightGBM": LGBMClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Extra Trees": ExtraTreesClassifier(),
    "Ridge Classifier": RidgeClassifier(),
    "Perceptron": Perceptron(),
    "GaussianNB": GaussianNB()

}

**Baseline**

In [4]:
df1=pd.read_csv("https://media.githubusercontent.com/media/shahriariit/opendataset/refs/heads/master/UniversalBank.csv")

In [5]:
df1.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [ ]:
print(df1['Personal Loan'].value_counts())

Personal Loan
0    4520
1     480
Name: count, dtype: int64


In [ ]:

X_df1 = df1.drop('Personal Loan', axis=1)
y_df1 = df1['Personal Loan']

X_train, X_test, y_train, y_test = train_test_split(X_df1, y_df1, test_size=0.2, random_state=42, stratify=y_df1)

In [ ]:
results_df1 = {}

print(f"\nEvaluating on df dataset with Train-Test Split\n{'='*40}")

for name, model in classifiers.items():
    print(f"Training {name} on df...")
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Calculate individual metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    gmean = g_mean(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    # Store results
    results_df1[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1-score': f1,
        'G-mean': gmean,
        'MCC': mcc,
    }

    # Print summary
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    print(f"G-Mean: {gmean:.4f}, MCC: {mcc:.4f}")
    print("-" * 40)

In [17]:
results_df1_table = pd.DataFrame(results_df1).T
display(results_df1_table.round(4))

,accuracy,precision,recall,f1-score,G-mean,MCC
Logistic Regression,0.936,0.7162,0.5521,0.6235,0.7343,0.5952
SVM,0.904,0.0000,0.0000,0.0000,0.0000,0.0000
Naive Bayes,0.878,0.4044,0.5729,0.4741,0.7222,0.4154
K-Nearest Neighbors,0.902,0.3750,0.0312,0.0577,0.1763,0.0850
Decision Tree,0.983,0.8762,0.9583,0.9154,0.9719,0.9071
Random Forest,0.991,0.9677,0.9375,0.9524,0.9666,0.9476
LightGBM,0.993,0.9684,0.9583,0.9634,0.9773,0.9595
AdaBoost,0.965,0.8765,0.7396,0.8023,0.8552,0.7866
Extra Trees,0.989,1.0000,0.8854,0.9392,0.9410,0.9353
Ridge Classifier,0.932,0.9118,0.3229,0.4769,0.5673,0.5195


**Clustered data**

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df1_scaled = scaler.fit_transform(df1)

dbscan = DBSCAN(eps=0.5, min_samples=5)
clusters = dbscan.fit_predict(df1_scaled)

df1['DBSCAN_Cluster'] = clusters

print("DBSCAN clustering applied. Cluster distribution:")
print(df1['DBSCAN_Cluster'].value_counts())

In [ ]:
df_riaz=pd.read_excel("https://github.com/mddruvo/DBscanYesNoNonNegative/blob/main/DBscanYesNoNonNegative.xlsx?raw=true", usecols=lambda col: not col.startswith('Unnamed'))

In [ ]:
df_riaz.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,DBSCAN_Cluster
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0,1
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0,2
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0,2
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1,0


In [ ]:
print(df_riaz['DBSCAN_Cluster'].unique())

[0 1 2 3 4]


In [ ]:
print(df_riaz['DBSCAN_Cluster'].value_counts())

DBSCAN_Cluster
2    1232
0    1043
3    1022
1     867
4     411
Name: count, dtype: int64
